In [ ]:
#|default_exp text_editor

# Text Editor

Implements functions for Anthropic's Text Editor Tool API, allowing Claude to view and edit files.

See the [Anthropic Text Editor Tool documentation](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/text-editor-tool) for details on the API commands (`view`, `create`, `insert`, `str_replace`, `undo_edit`) and usage. The `str_replace_editor` function acts as the required dispatcher.

In [ ]:
#| export
from pathlib import Path

In [ ]:
#| hide
from claudette.core import *
from toolslm.funccall import *

In [ ]:
#| exports
def view(path:str, view_range:tuple[int,int]=None, nums:bool=False):
    'View directory or file contents with optional line range and numbers'
    try:
        p = Path(path).expanduser().resolve()
        if not p.exists(): return f'Error: File not found: {p}'
        if p.is_dir():
            res = [str(f) for f in p.glob('**/*') 
                   if not any(part.startswith('.') for part in f.relative_to(p).parts)]
            return f'Directory contents of {p}:\n' + '\n'.join(res)
        
        lines = p.read_text().splitlines()
        s,e = 1,len(lines)
        if view_range:
            s,e = view_range
            if not (1 <= s <= len(lines)): return f'Error: Invalid start line {s}'
            if e != -1 and not (s <= e <= len(lines)): return f'Error: Invalid end line {e}'
            lines = lines[s-1:None if e==-1 else e]
            
        return '\n'.join([f'{i+s-1:6d} │ {l}' for i,l in enumerate(lines,1)] if nums else lines)
    except Exception as e: return f'Error viewing file: {str(e)}'

In [ ]:
print(view('_quarto.yml', (1,10), nums=True))

     1 │ project:
     2 │   type: website
     3 │   resources: 
     4 │     - "*.txt"
     5 │   preview:
     6 │     port: 3000
     7 │     browser: false
     8 │ 
     9 │ format:
    10 │   html:


In [ ]:
#| exports
def create(path: str, file_text: str, overwrite:bool=False) -> str:
    'Creates a new file with the given content at the specified path'
    try:
        p = Path(path)
        if p.exists():
            if not overwrite: return f'Error: File already exists: {p}'
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text(file_text)
        return f'Created file {p} containing:\n{file_text}'
    except Exception as e: return f'Error creating file: {str(e)}'

In [ ]:
print(create('test.txt', 'Hello, world!'))
print(view('test.txt', nums=True))

Error: File already exists: test.txt
     1 │ Let's add a 
     2 │ Let's add a 
     3 │ Let's add a 
     4 │ Hello, world!


In [ ]:
#| exports
def insert(path: str, insert_line: int, new_str: str) -> str:
    'Insert new_str at specified line number'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
            
        content = p.read_text().splitlines()
        if not (0 <= insert_line <= len(content)): return f'Error: Invalid line number {insert_line}'
            
        content.insert(insert_line, new_str)
        new_content = '\n'.join(content)
        p.write_text(new_content)
        return f'Inserted text at line {insert_line} in {p}.\nNew contents:\n{new_content}'
    except Exception as e: return f'Error inserting text: {str(e)}'

In [ ]:
insert('test.txt', 0, 'Let\'s add a new line')
print(view('test.txt', nums=True))

     1 │ Let's add a new line
     2 │ Let's add a 
     3 │ Let's add a 
     4 │ Let's add a 
     5 │ Hello, world!


In [ ]:
#| exports
def str_replace(path: str, old_str: str, new_str: str) -> str:
    'Replace first occurrence of old_str with new_str in file'
    try:
        p = Path(path)
        if not p.exists(): return f'Error: File not found: {p}'
            
        content = p.read_text()
        count = content.count(old_str)
        
        if count == 0: return 'Error: Text not found in file'
        if count > 1: return f'Error: Multiple matches found ({count})'
            
        new_content = content.replace(old_str, new_str, 1)
        p.write_text(new_content)
        return f'Replaced text in {p}.\nNew contents:\n{new_content}'
    except Exception as e: return f'Error replacing text: {str(e)}'

In [ ]:
str_replace('test.txt', 'new line', '')
print(view('test.txt', nums=True))

     1 │ Let's add a 
     2 │ Let's add a 
     3 │ Let's add a 
     4 │ Let's add a 
     5 │ Hello, world!


In [ ]:
#| exports
def str_replace_editor(**kwargs: dict[str, str]) -> str:
    'Dispatcher for Anthropic Text Editor tool commands: view, str_replace, create, insert, undo_edit.'
    cmd = kwargs.pop('command', '')
    if cmd == 'view': return view(**kwargs)
    elif cmd == 'str_replace': return str_replace(**kwargs) 
    elif cmd == 'create': return create(**kwargs)
    elif cmd == 'insert': return insert(**kwargs)
    elif cmd == 'undo_edit': return 'Undo edit is not supported yet.'
    else: return f'Unknown command: {cmd}'

def str_replace_based_edit_tool(**kwargs: dict[str, str]) -> str:
    'Dispatcher for Anthropic Text Editor tool commands: view, str_replace, create, insert.'
    return str_replace_editor(**kwargs)

In [ ]:
#| export
text_editor_conf = {
    'sonnet': {"type": "text_editor_20250429", "name": "str_replace_based_edit_tool"},
    'sonnet-3-7': {'type': 'text_editor_20250124', 'name': 'str_replace_editor'},
    'sonnet-3-5': {'type': 'text_editor_20241022', 'name': 'str_replace_editor'}
}

In [ ]:
model = models[1]

In [ ]:
c = Chat(model, tools=[text_editor_conf['sonnet']], ns=mk_ns(str_replace_based_edit_tool))
c.toolloop('Please explain what my _quarto.yml does. Use your tools')

## Explanation of Your `_quarto.yml` File

Your `_quarto.yml` file is a configuration file for **Quarto**, a scientific and technical publishing system. This file is specifically set up to generate a documentation website for your `claudette` project. Here's what each section does:

### **Project Configuration**
```yaml
project:
  type: website
  resources: 
    - "*.txt"
  preview:
    port: 3000
    browser: false
```
- **`type: website`**: Tells Quarto to build a website (rather than a book, document, etc.)
- **`resources`**: Includes all `.txt` files in the website output
- **`preview`**: Sets up local development server on port 3000 without auto-opening browser

### **HTML Format Settings**
```yaml
format:
  html:
    theme: cosmo
    css: styles.css
    toc: true
    code-tools: true
    code-block-bg: true
    code-block-border-left: "#31BAE9"
    highlight-style: arrow
```
- **`theme: cosmo`**: Uses the "Cosmo" Bootstrap theme for styling
- **`css: styles.css`**: Applies custom CSS from your `styles.css` file
- **`toc: true`**: Enables table of contents on pages
- **`code-tools: true`**: Adds interactive code tools (copy, view source, etc.)
- **Code styling**: Sets background, blue left border (#31BAE9), and "arrow" syntax highlighting

### **Layout Configuration**
```yaml
grid:
  sidebar-width: 180px
  body-width: 1800px
  margin-width: 150px
  gutter-width: 1.0rem
keep-md: true
```
- **Grid system**: Defines a wide layout (1800px body) with specific sidebar and margin widths
- **`keep-md: true`**: Preserves intermediate Markdown files during rendering

### **Website Features**
```yaml
website:
  twitter-card: true
  open-graph: true
  repo-actions: [issue]
  navbar:
    background: primary
    search: true
  sidebar:
    style: floating
```
- **Social media**: Enables Twitter cards and Open Graph meta tags for link previews
- **`repo-actions: [issue]`**: Adds "Report an issue" link to pages
- **Navigation**: Primary-colored navbar with search functionality
- **`sidebar: floating`**: Creates a floating sidebar style

### **Metadata Integration**
```yaml
metadata-files:
  - nbdev.yml
  - sidebar.yml
```
- Imports additional configuration from `nbdev.yml` (which I can see contains site title, URL, and repo info)
- References `sidebar.yml` (though this file doesn't exist in your directory)

### **Purpose**
This configuration is designed for **nbdev**, a system that creates documentation websites from Jupyter notebooks. It's set up to:
1. Convert your Jupyter notebooks (like `00_core.ipynb`, `01_toolloop.ipynb`, etc.) into web pages
2. Create a professional documentation site for the `claudette` library
3. Publish to `https://claudette.answer.ai/` (as specified in `nbdev.yml`)
4. Provide a wide layout suitable for code documentation with syntax highlighting and interactive features

The setup suggests this is a Python library documentation site built from Jupyter notebooks using the nbdev workflow.

<details>

- id: `msg_01RYHyNnmymn6hBXMHQNS1ck`
- content: `[{'citations': None, 'text': '## Explanation of Your `_quarto.yml` File\n\nYour `_quarto.yml` file is a configuration file for **Quarto**, a scientific and technical publishing system. This file is specifically set up to generate a documentation website for your `claudette` project. Here\'s what each section does:\n\n### **Project Configuration**\n```yaml\nproject:\n  type: website\n  resources: \n    - "*.txt"\n  preview:\n    port: 3000\n    browser: false\n```\n- **`type: website`**: Tells Quarto to build a website (rather than a book, document, etc.)\n- **`resources`**: Includes all `.txt` files in the website output\n- **`preview`**: Sets up local development server on port 3000 without auto-opening browser\n\n### **HTML Format Settings**\n```yaml\nformat:\n  html:\n    theme: cosmo\n    css: styles.css\n    toc: true\n    code-tools: true\n    code-block-bg: true\n    code-block-border-left: "#31BAE9"\n    highlight-style: arrow\n```\n- **`theme: cosmo`**: Uses the "Cosmo" Bootstrap theme for styling\n- **`css: styles.css`**: Applies custom CSS from your `styles.css` file\n- **`toc: true`**: Enables table of contents on pages\n- **`code-tools: true`**: Adds interactive code tools (copy, view source, etc.)\n- **Code styling**: Sets background, blue left border (#31BAE9), and "arrow" syntax highlighting\n\n### **Layout Configuration**\n```yaml\ngrid:\n  sidebar-width: 180px\n  body-width: 1800px\n  margin-width: 150px\n  gutter-width: 1.0rem\nkeep-md: true\n```\n- **Grid system**: Defines a wide layout (1800px body) with specific sidebar and margin widths\n- **`keep-md: true`**: Preserves intermediate Markdown files during rendering\n\n### **Website Features**\n```yaml\nwebsite:\n  twitter-card: true\n  open-graph: true\n  repo-actions: [issue]\n  navbar:\n    background: primary\n    search: true\n  sidebar:\n    style: floating\n```\n- **Social media**: Enables Twitter cards and Open Graph meta tags for link previews\n- **`repo-actions: [issue]`**: Adds "Report an issue" link to pages\n- **Navigation**: Primary-colored navbar with search functionality\n- **`sidebar: floating`**: Creates a floating sidebar style\n\n### **Metadata Integration**\n```yaml\nmetadata-files:\n  - nbdev.yml\n  - sidebar.yml\n```\n- Imports additional configuration from `nbdev.yml` (which I can see contains site title, URL, and repo info)\n- References `sidebar.yml` (though this file doesn\'t exist in your directory)\n\n### **Purpose**\nThis configuration is designed for **nbdev**, a system that creates documentation websites from Jupyter notebooks. It\'s set up to:\n1. Convert your Jupyter notebooks (like `00_core.ipynb`, `01_toolloop.ipynb`, etc.) into web pages\n2. Create a professional documentation site for the `claudette` library\n3. Publish to `https://claudette.answer.ai/` (as specified in `nbdev.yml`)\n4. Provide a wide layout suitable for code documentation with syntax highlighting and interactive features\n\nThe setup suggests this is a Python library documentation site built from Jupyter notebooks using the nbdev workflow.', 'type': 'text'}]`
- model: `claude-sonnet-4-20250514`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 3525, 'output_tokens': 826, 'server_tool_use': None, 'service_tier': 'standard'}`
- steps: `[{'role': 'assistant', 'content': [{'citations': None, 'text': "I'll help you understand what your `_quarto.yml` file does. Let me first locate and examine the file.", 'type': 'text'}, {'id': 'toolu_01WKRWYwHSGvKDcnsGViKKic', 'input': {'command': 'view', 'path': '.'}, 'name': 'str_replace_based_edit_tool', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01WKRWYwHSGvKDcnsGViKKic', 'content': 'Directory contents of /Users/jhoward/aai-ws/claudette:\n/Users/jhoward/aai-ws/claudette/claudette\n/Users/jhoward/aai-ws/claudette/llms.txt\n/Users/jhoward/aai-ws/claudette/00_core.ipynb\n/Users/jhoward/aai-ws/claudette/_quarto.yml\n/Users/jhoward/aai-ws/claudette/claudette.egg-info\n/Users/jhoward/aai-ws/claudette/tools\n/Users/jhoward/aai-ws/claudette/LICENSE\n/Users/jhoward/aai-ws/claudette/styles.css\n/Users/jhoward/aai-ws/claudette/CHANGELOG.md\n/Users/jhoward/aai-ws/claudette/dist\n/Users/jhoward/aai-ws/claudette/CNAME\n/Users/jhoward/aai-ws/claudette/llms-ctx-full.txt\n/Users/jhoward/aai-ws/claudette/pyproject.toml\n/Users/jhoward/aai-ws/claudette/02_async.ipynb\n/Users/jhoward/aai-ws/claudette/MANIFEST.in\n/Users/jhoward/aai-ws/claudette/apilist.txt\n/Users/jhoward/aai-ws/claudette/03_text_editor.ipynb\n/Users/jhoward/aai-ws/claudette/README.md\n/Users/jhoward/aai-ws/claudette/samples\n/Users/jhoward/aai-ws/claudette/setup.py\n/Users/jhoward/aai-ws/claudette/01_toolloop.ipynb\n/Users/jhoward/aai-ws/claudette/nbdev.yml\n/Users/jhoward/aai-ws/claudette/llms-ctx.txt\n/Users/jhoward/aai-ws/claudette/README.txt\n/Users/jhoward/aai-ws/claudette/settings.ini\n/Users/jhoward/aai-ws/claudette/test.txt\n/Users/jhoward/aai-ws/claudette/build\n/Users/jhoward/aai-ws/claudette/CHANGELOG.bak\n/Users/jhoward/aai-ws/claudette/index.ipynb\n/Users/jhoward/aai-ws/claudette/claudette/toolloop.py\n/Users/jhoward/aai-ws/claudette/claudette/_modidx.py\n/Users/jhoward/aai-ws/claudette/claudette/__init__.py\n/Users/jhoward/aai-ws/claudette/claudette/core.py\n/Users/jhoward/aai-ws/claudette/claudette/__pycache__\n/Users/jhoward/aai-ws/claudette/claudette/text_editor.py\n/Users/jhoward/aai-ws/claudette/claudette/asink.py\n/Users/jhoward/aai-ws/claudette/claudette.egg-info/PKG-INFO\n/Users/jhoward/aai-ws/claudette/claudette.egg-info/not-zip-safe\n/Users/jhoward/aai-ws/claudette/claudette.egg-info/SOURCES.txt\n/Users/jhoward/aai-ws/claudette/claudette.egg-info/entry_points.txt\n/Users/jhoward/aai-ws/claudette/claudette.egg-info/requires.txt\n/Users/jhoward/aai-ws/claudette/claudette.egg-info/top_level.txt\n/Users/jhoward/aai-ws/claudette/claudette.egg-info/dependency_links.txt\n/Users/jhoward/aai-ws/claudette/tools/refresh_llm_docs.sh\n/Users/jhoward/aai-ws/claudette/dist/claudette-0.1.10-py3-none-any.whl\n/Users/jhoward/aai-ws/claudette/dist/claudette-0.1.10.tar.gz\n/Users/jhoward/aai-ws/claudette/samples/puppy.jpg\n/Users/jhoward/aai-ws/claudette/build/bdist.macosx-11.0-arm64\n/Users/jhoward/aai-ws/claudette/build/lib\n/Users/jhoward/aai-ws/claudette/claudette/__pycache__/toolloop.cpython-312.pyc\n/Users/jhoward/aai-ws/claudette/claudette/__pycache__/_modidx.cpython-312.pyc\n/Users/jhoward/aai-ws/claudette/claudette/__pycache__/core.cpython-312.pyc\n/Users/jhoward/aai-ws/claudette/claudette/__pycache__/text_editor.cpython-312.pyc\n/Users/jhoward/aai-ws/claudette/claudette/__pycache__/asink.cpython-312.pyc\n/Users/jhoward/aai-ws/claudette/claudette/__pycache__/__init__.cpython-312.pyc\n/Users/jhoward/aai-ws/claudette/build/lib/claudette\n/Users/jhoward/aai-ws/claudette/build/lib/claudette/toolloop.py\n/Users/jhoward/aai-ws/claudette/build/lib/claudette/_modidx.py\n/Users/jhoward/aai-ws/claudette/build/lib/claudette/__init__.py\n/Users/jhoward/aai-ws/claudette/build/lib/claudette/core.py\n/Users/jhoward/aai-ws/claudette/build/lib/claudette/text_editor.py\n/Users/jhoward/aai-ws/claudette/build/lib/claudette/asink.py'}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': 'Now let me examine the `_quarto.yml` file:', 'type': 'text'}, {'id': 'toolu_014rHW8HsYQRQpgkAW3DkpEW', 'input': {'command': 'view', 'path': '/_Users/jhoward/aai-ws/claudette/_quarto.yml'}, 'name': 'str_replace_based_edit_tool', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_014rHW8HsYQRQpgkAW3DkpEW', 'content': 'Error: File not found: /_Users/jhoward/aai-ws/claudette/_quarto.yml'}]}, {'role': 'assistant', 'content': [{'id': 'toolu_012hEVqDyF9h3FRBczTz379x', 'input': {'command': 'view', 'path': '/Users/jhoward/aai-ws/claudette/_quarto.yml'}, 'name': 'str_replace_based_edit_tool', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_012hEVqDyF9h3FRBczTz379x', 'content': 'project:\n  type: website\n  resources: \n    - "*.txt"\n  preview:\n    port: 3000\n    browser: false\n\nformat:\n  html:\n    theme: cosmo\n    css: styles.css\n    toc: true\n    code-tools: true\n    code-block-bg: true\n    code-block-border-left: "#31BAE9"\n    highlight-style: arrow\n    grid:\n      sidebar-width: 180px\n      body-width: 1800px\n      margin-width: 150px\n      gutter-width: 1.0rem\n    keep-md: true\n  commonmark: default\n\nwebsite:\n  twitter-card: true\n  open-graph: true\n  repo-actions: [issue]\n  navbar:\n    background: primary\n    search: true\n  sidebar:\n    style: floating\n\nmetadata-files:\n  - nbdev.yml\n  - sidebar.yml\n'}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': 'Now let me also check if there are any related configuration files to better understand the context:', 'type': 'text'}, {'id': 'toolu_01DfXFLrS1auNy2EyoXRCDiV', 'input': {'command': 'view', 'path': '/Users/jhoward/aai-ws/claudette/nbdev.yml'}, 'name': 'str_replace_based_edit_tool', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_01DfXFLrS1auNy2EyoXRCDiV', 'content': 'project:\n  output-dir: _docs\n\nwebsite:\n  title: "claudette"\n  site-url: "https://claudette.answer.ai/"\n  description: "Claudette is Claude\'s friend"\n  repo-branch: main\n  repo-url: "https://github.com/AnswerDotAI/claudette"'}]}, {'role': 'assistant', 'content': [{'id': 'toolu_0171znvzm9oNw2ApYGhhFfrL', 'input': {'command': 'view', 'path': '/Users/jhoward/aai-ws/claudette/sidebar.yml'}, 'name': 'str_replace_based_edit_tool', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_0171znvzm9oNw2ApYGhhFfrL', 'content': 'Error: File not found: /Users/jhoward/aai-ws/claudette/sidebar.yml'}]}, {'role': 'assistant', 'content': [{'citations': None, 'text': '## Explanation of Your `_quarto.yml` File\n\nYour `_quarto.yml` file is a configuration file for **Quarto**, a scientific and technical publishing system. This file is specifically set up to generate a documentation website for your `claudette` project. Here\'s what each section does:\n\n### **Project Configuration**\n```yaml\nproject:\n  type: website\n  resources: \n    - "*.txt"\n  preview:\n    port: 3000\n    browser: false\n```\n- **`type: website`**: Tells Quarto to build a website (rather than a book, document, etc.)\n- **`resources`**: Includes all `.txt` files in the website output\n- **`preview`**: Sets up local development server on port 3000 without auto-opening browser\n\n### **HTML Format Settings**\n```yaml\nformat:\n  html:\n    theme: cosmo\n    css: styles.css\n    toc: true\n    code-tools: true\n    code-block-bg: true\n    code-block-border-left: "#31BAE9"\n    highlight-style: arrow\n```\n- **`theme: cosmo`**: Uses the "Cosmo" Bootstrap theme for styling\n- **`css: styles.css`**: Applies custom CSS from your `styles.css` file\n- **`toc: true`**: Enables table of contents on pages\n- **`code-tools: true`**: Adds interactive code tools (copy, view source, etc.)\n- **Code styling**: Sets background, blue left border (#31BAE9), and "arrow" syntax highlighting\n\n### **Layout Configuration**\n```yaml\ngrid:\n  sidebar-width: 180px\n  body-width: 1800px\n  margin-width: 150px\n  gutter-width: 1.0rem\nkeep-md: true\n```\n- **Grid system**: Defines a wide layout (1800px body) with specific sidebar and margin widths\n- **`keep-md: true`**: Preserves intermediate Markdown files during rendering\n\n### **Website Features**\n```yaml\nwebsite:\n  twitter-card: true\n  open-graph: true\n  repo-actions: [issue]\n  navbar:\n    background: primary\n    search: true\n  sidebar:\n    style: floating\n```\n- **Social media**: Enables Twitter cards and Open Graph meta tags for link previews\n- **`repo-actions: [issue]`**: Adds "Report an issue" link to pages\n- **Navigation**: Primary-colored navbar with search functionality\n- **`sidebar: floating`**: Creates a floating sidebar style\n\n### **Metadata Integration**\n```yaml\nmetadata-files:\n  - nbdev.yml\n  - sidebar.yml\n```\n- Imports additional configuration from `nbdev.yml` (which I can see contains site title, URL, and repo info)\n- References `sidebar.yml` (though this file doesn\'t exist in your directory)\n\n### **Purpose**\nThis configuration is designed for **nbdev**, a system that creates documentation websites from Jupyter notebooks. It\'s set up to:\n1. Convert your Jupyter notebooks (like `00_core.ipynb`, `01_toolloop.ipynb`, etc.) into web pages\n2. Create a professional documentation site for the `claudette` library\n3. Publish to `https://claudette.answer.ai/` (as specified in `nbdev.yml`)\n4. Provide a wide layout suitable for code documentation with syntax highlighting and interactive features\n\nThe setup suggests this is a Python library documentation site built from Jupyter notebooks using the nbdev workflow.', 'type': 'text'}]}]`

</details>